In [7]:
import json
import pandas as pd
import os
import shutil
import os
import xml.etree.cElementTree as ET
from PIL import Image
from math import floor

In [8]:
f = open('kavsir_bboxes.json')
labels = json.load(f)

In [9]:
df = pd.DataFrame.from_dict(labels, orient='index').reset_index(level=0).rename(columns ={'index':'image'})

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
df_train, df_val = train_test_split(df, random_state=10)

In [12]:
df_train.reset_index(drop = True, inplace = True)
df_val.reset_index(drop = True, inplace = True)

In [15]:
def create_root(file_prefix, width, height):
    root = ET.Element("annotations")
    ET.SubElement(root, "filename").text = "{}.jpg".format(file_prefix)
    ET.SubElement(root, "folder").text = "images"
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = "3"
    return root


def create_object_annotation(root, voc_labels):
    for voc_label in voc_labels:
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text = "Polyps"
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = str(0)
        ET.SubElement(obj, "difficult").text = str(0)
        bbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bbox, "xmin").text = str(voc_label['xmin'])
        ET.SubElement(bbox, "ymin").text = str(voc_label['ymin'])
        ET.SubElement(bbox, "xmax").text = str(voc_label['xmax'])
        ET.SubElement(bbox, "ymax").text = str(voc_label['ymax'])
    return root


def create_file(file_prefix, width, height, voc_labels,typ):
    root = create_root(file_prefix, width, height)
    root = create_object_annotation(root, voc_labels)
    tree = ET.ElementTree(root)
    tree.write("{}/{}.xml".format('images/'+typ+"/", file_prefix))

In [16]:
main_dir = 'images/'
for typ in ['train','test']:
    if typ == 'train':
        dff = df_train
    else:
        dff = df_val
    for i in dff.index:
        shutil.copy(main_dir+df_train.iloc[i,0]+'.jpg',main_dir+typ+'/'+df_train.iloc[i,0]+'.jpg')
        create_file(df_train.iloc[i,0], df_train.iloc[i,2], df_train.iloc[i,1], df_train.iloc[i,3], typ)